# 6.3 InceptionV1

### 导入utils中的DataLoader

In [1]:
from utils import *

In [2]:
class Inception_cell(nn.Module):
    def __init__(self, in_channel, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception_cell, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_channel, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )

        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_channel, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )

        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_channel, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )

        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_channel, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1,y2,y3,y4], 1) ## 将四个分支并联起来

In [3]:
class GoogLeNet(nn.Module):
    def __init__(self):
        super(GoogLeNet, self).__init__()
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )
        
        self.a3 = Inception_cell(192,  64,  96, 128, 16, 32, 32)
        self.b3 = Inception_cell(256, 128, 128, 192, 32, 96, 64)

        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.a4 = Inception_cell(480, 192,  96, 208, 16,  48,  64)
        self.b4 = Inception_cell(512, 160, 112, 224, 24,  64,  64)
        self.c4 = Inception_cell(512, 128, 128, 256, 24,  64,  64)
        self.d4 = Inception_cell(512, 112, 144, 288, 32,  64,  64)
        self.e4 = Inception_cell(528, 256, 160, 320, 32, 128, 128)

        self.a5 = Inception_cell(832, 256, 160, 320, 32, 128, 128)
        self.b5 = Inception_cell(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.linear = nn.Linear(1024, 10)

    def forward(self, x):
        out = self.pre_layers(x)
        out = self.a3(out)
        out = self.b3(out)
        out = self.maxpool(out)
        out = self.a4(out)
        out = self.b4(out)
        out = self.c4(out)
        out = self.d4(out)
        out = self.e4(out)
        out = self.maxpool(out)
        out = self.a5(out)
        out = self.b5(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
## 验证下构型结构是否构造正确
net = GoogLeNet()
x = torch.randn(1, 3, 32, 32)
y = net(x)
print(y.size())

torch.Size([1, 10])


借助PyTorch内置的nn.Sequential类构建模型。

In [5]:
from torchsummary import summary
## 需要预先下载，在终端输入 pip install torchsummary
IMSIZE = 32
inceptionv1_model = GoogLeNet().cuda()
summary(inceptionv1_model, (3, IMSIZE, IMSIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 192, 32, 32]           5,376
       BatchNorm2d-2          [-1, 192, 32, 32]             384
              ReLU-3          [-1, 192, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          12,352
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 96, 32, 32]          18,528
       BatchNorm2d-8           [-1, 96, 32, 32]             192
              ReLU-9           [-1, 96, 32, 32]               0
           Conv2d-10          [-1, 128, 32, 32]         110,720
      BatchNorm2d-11          [-1, 128, 32, 32]             256
             ReLU-12          [-1, 128, 32, 32]               0
           Conv2d-13           [-1, 16, 32, 32]           3,088
      BatchNorm2d-14           [-1, 16,

## 模型训练

先定义几个训练中会用到的函数。

输入参数epochs和lr，实施训练。

In [6]:
lr = 1e-4
optimizer = torch.optim.Adam(inceptionv1_model.parameters(), lr=lr)  # 设置优化器
epochs = 1
train_loader, val_loader = get_cifar10(batch_size=32)
history = train(inceptionv1_model, optimizer, train_loader, val_loader, epochs=epochs)  # 实施训练

Epoch [1/1], time: 177.90s, loss: 1.1286, acc: 0.5946, val_loss: 0.8535, val_acc: 0.7035


## 模型预测

从训练集中选取几张图片，使用模型做出预测，观察结果。

In [7]:
testiter = iter(val_loader)

每次对迭代器testiter执行next，就会取出其下一个元素。因此，多次运行以下代码块，即可测试不同的图片。

In [7]:
from matplotlib import pyplot as plt
image = next(testiter)[0][0]
plt.axis('off')
plt.imshow(image.permute((1, 2, 0)))
plt.show()

pred = predict(inceptionv1_model, image.unsqueeze(dim=0))  # 由于模型接受的输入shape为4维（第1维为batch_size），因此需要使用unsqueeze函数在图片前面加一维
val_set = torchvision.datasets.CIFAR10(root='../dataset',train=False)
print('Prediction:', val_set.classes[pred])

NameError: name 'testiter' is not defined